In [ ]:
elastic_rods_dir = '../../elastic_rods/python/'
weaving_dir = '../'
import os.path as osp
import sys; sys.path.append(elastic_rods_dir); sys.path.append(weaving_dir)
import numpy as np, elastic_rods, linkage_vis
import numpy.linalg as la
from bending_validation import suppress_stdout as so
import matplotlib.pyplot as plt
import copy
‰
import vis.fields
import matplotlib.cm as cm

# Reloadable libraries
import analysis_helper, ribbon_linkage_helper, mesh_vis, linkage_utils, compute_curve_from_curvature, importlib
importlib.reload(analysis_helper)
importlib.reload(ribbon_linkage_helper)
importlib.reload(mesh_vis)
importlib.reload(linkage_utils)
importlib.reload(compute_curve_from_curvature)
from analysis_helper import (compare_turning_angle,
                            is_on_sphere, 
                            get_distance_to_center_scalar_field, 
                            plot_curvatures, 
                            get_curvature_scalar_field,
                            construct_elastic_rod_loop_from_rod_segments, 
                            concatenate_rod_properties_from_rod_segments, 
                            compute_min_distance_rigid_transformation)
from ribbon_linkage_helper import (initialize_linkage, 
                                   update_rest_curvature, 
                                   set_ribbon_linkage,
                                   export_linkage_geometry_to_obj)
from compute_curve_from_curvature import (match_geo_curvature_and_edge_len, get_all_curve_pattern)
from linkage_utils import order_segments_by_ribbons, get_turning_angle_and_length_from_ordered_rods

In [ ]:
# Sphere Parameters
default_camera_parameters = ((3.466009282140468, -4.674139805388271, -2.556131049738206), (-0.21402574298422497, -0.06407538766530313, -0.9747681088523519),(0.1111, 0.1865, 0.5316))
RIBBON_CS = [1, 10]
ISO_CS = [4, 4]
MODEL_PATH = osp.join(weaving_dir + 'models/equal_edge_atI.obj')
SUBDIVISION_RESOLUTION = 7
REGULARIZATION_WEIGHT = 0.05
SMOOTHING_WEIGHT = 10
RIBBON_NAME = 'sphere_strip'

In [ ]:
# Torus Parameters
default_camera_parameters = ((3.466009282140468, -4.674139805388271, -2.556131049738206), (-0.21402574298422497, -0.06407538766530313, -0.9747681088523519),(0.1111, 0.1865, 0.5316))
RIBBON_CS = [0.1, 4]
ISO_CS = [1, 1]
MODEL_PATH = osp.join(weaving_dir + 'models/tri_hex_torus_linkage.obj')
SUBDIVISION_RESOLUTION = 20
REGULARIZATION_WEIGHT = 0
SMOOTHING_WEIGHT = 1
RIBBON_NAME = 'torus_strip'

In [ ]:
# # Sphere to peanut sequence 1 Parameters
# default_camera_parameters = ((3.466009282140468, -4.674139805388271, -2.556131049738206), (-0.21402574298422497, -0.06407538766530313, -0.9747681088523519),(0.1111, 0.1865, 0.5316))
# RIBBON_CS = [0.01, 0.1]
# ISO_CS = [0.04, 0.04]
# MODEL_PATH = osp.join(weaving_dir + 'models/sphere_to_peanut/1.obj')
# SUBDIVISION_RESOLUTION = 20
# REGULARIZATION_WEIGHT = 0.05

In [ ]:
# Sphere to hemoglobin sequence
default_camera_parameters = ((3.466009282140468, -4.674139805388271, -2.556131049738206), (-0.21402574298422497, -0.06407538766530313, -0.9747681088523519),(0.1111, 0.1865, 0.5316))
RIBBON_CS = [0.01, 0.1]
ISO_CS = [0.04, 0.04]
MODEL_PATH = osp.join(weaving_dir + 'models/sphere_to_hemoglobin/5.obj')
SUBDIVISION_RESOLUTION = 20
REGULARIZATION_WEIGHT = 0.0
MODEL_NAME = './results/sphere_to_hemoglobin_5'

In [ ]:
# Sphere to simple hemoglobin sequence
default_camera_parameters = ((3.466009282140468, -4.674139805388271, -2.556131049738206), (-0.21402574298422497, -0.06407538766530313, -0.9747681088523519),(0.1111, 0.1865, 0.5316))
RIBBON_CS = [0.01, 0.5]
ISO_CS = [0.04, 0.04]
MODEL_PATH = osp.join(weaving_dir + 'models/simple_sphere_to_hemoglobin/5.obj')
SUBDIVISION_RESOLUTION = 20
REGULARIZATION_WEIGHT = 0.001
SMOOTHING_WEIGHT = 0
MODEL_NAME = './results/simple_sphere_to_hemoglobin'
RIBBON_NAME = 'simple_hemoglobin_strip'

In [ ]:
# GMO Torus Parameters
default_camera_parameters = ((3.466009282140468, -4.674139805388271, -2.556131049738206), (-0.21402574298422497, -0.06407538766530313, -0.9747681088523519),(0.1111, 0.1865, 0.5316))
RIBBON_CS = [1, 40]
ISO_CS = [1, 1]
MODEL_PATH = osp.join(weaving_dir + 'models/gmo_torus/gmo_torus.obj')
MODEL_NAME = 'gmo_torus'
SUBDIVISION_RESOLUTION = 20
REGULARIZATION_WEIGHT = 0
SMOOTHING_WEIGHT = 0.001
RIBBON_NAME = 'gmo_torus_strip'

In [ ]:
# Regular Torus Parameters
default_camera_parameters = ((3.466009282140468, -4.674139805388271, -2.556131049738206), (-0.21402574298422497, -0.06407538766530313, -0.9747681088523519),(0.1111, 0.1865, 0.5316))
RIBBON_CS = [1, 40]
ISO_CS = [1, 1]
MODEL_PATH = osp.join(weaving_dir + 'models/regular_torus.obj')
MODEL_NAME = 'regular_torus'
SUBDIVISION_RESOLUTION = 20
SMOOTHING_WEIGHT = 0.001
REGULARIZATION_WEIGHT = 0
RIBBON_NAME = 'regular_torus_strip'

In [ ]:
# # Pseudosphere
# default_camera_parameters = ((3.466009282140468, -4.674139805388271, -2.556131049738206), (-0.21402574298422497, -0.06407538766530313, -0.9747681088523519),(0.1111, 0.1865, 0.5316))
# RIBBON_CS = [0.001, 0.01]
# ISO_CS = [1, 1]
# MODEL_NAME = "pseudo_sphere"
# MODEL_PATH = osp.join(weaving_dir + 'models/{}.obj'.format(MODEL_NAME))
# SUBDIVISION_RESOLUTION = 20
# SMOOTHING_WEIGHT = 0.0000001
# REGULARIZATION_WEIGHT = 0
# RIBBON_NAME = "pseudo_sphere_strip"

In [ ]:
# # Pseudosphere 400
# default_camera_parameters = ((3.466009282140468, -4.674139805388271, -2.556131049738206), (-0.21402574298422497, -0.06407538766530313, -0.9747681088523519),(0.1111, 0.1865, 0.5316))
# RIBBON_CS = [0.1, 4]
# ISO_CS = [1, 1]
# MODEL_NAME = "pseudo_sphere_400"
# MODEL_PATH = osp.join(weaving_dir + 'models/{}.obj'.format(MODEL_NAME))
# SUBDIVISION_RESOLUTION = 20
# SMOOTHING_WEIGHT = 100
# REGULARIZATION_WEIGHT = 1
# RIBBON_NAME = "pseudo_sphere_400_strip"

In [ ]:
# Pseudosphere Small
default_camera_parameters = ((3.466009282140468, -4.674139805388271, -2.556131049738206), (-0.21402574298422497, -0.06407538766530313, -0.9747681088523519),(0.1111, 0.1865, 0.5316))
RIBBON_CS = [0.005, 0.03]
ISO_CS = [1, 1]
MODEL_NAME = "small_pseudo_sphere"
MODEL_PATH = osp.join(weaving_dir + 'models/{}.obj'.format(MODEL_NAME))
SUBDIVISION_RESOLUTION = 20
SMOOTHING_WEIGHT = 0.001
REGULARIZATION_WEIGHT = 0
RIBBON_NAME = "small_pseudo_sphere_strip"

In [ ]:
def initialize_curved_linkage(cross_section = ISO_CS, subdivision_res = SUBDIVISION_RESOLUTION, model_path = MODEL_PATH, cam_param = default_camera_parameters, regularization_weight = 0.1, smoothing_weight = 1):
    l = elastic_rods.RodLinkage(model_path, subdivision_res, False)
    driver=l.centralJoint()
    l.setMaterial(elastic_rods.RodMaterial('rectangle', 2000, 0.3, cross_section, stiffAxis=elastic_rods.StiffAxis.D1))
    l.set_design_parameter_config(use_restLen = True, use_restKappa = True)
    design_opts = elastic_rods.NewtonOptimizerOptions()
    design_opts.niter = 20
    design_opts.verbose = 10
    elastic_rods.designParameter_solve(l, design_opts, regularization_weight = regularization_weight, smoothing_weight = smoothing_weight)
    jdo = l.dofOffsetForJoint(driver)
    fixedVars = list(range(jdo, jdo + 6)) # fix rigid motion for a single joint
    eqm_opts = elastic_rods.NewtonOptimizerOptions()
    eqm_opts.niter = 400
    elastic_rods.compute_equilibrium(l, fixedVars=fixedVars, options = eqm_opts)
    view = linkage_vis.LinkageViewer(l, width=1024, height=640)
    view.setCameraParams(cam_param)
    return l, view

In [ ]:
with so(): linkage, view = initialize_curved_linkage(model_path = MODEL_PATH, cam_param = default_camera_parameters, cross_section = RIBBON_CS, subdivision_res = SUBDIVISION_RESOLUTION, regularization_weight = REGULARIZATION_WEIGHT, smoothing_weight=SMOOTHING_WEIGHT)
view.show()

In [ ]:
plt.plot(linkage.getRestKappaVars()[:SUBDIVISION_RESOLUTION * 1 - 1], '-d')
plt.savefig('rest_curvature.png')

In [ ]:
with so():
    jdo = linkage.dofOffsetForJoint(0)
    fixedVars = list(range(jdo, jdo + 6)) # fix rigid motion for a single joint
    eqm_opts = elastic_rods.NewtonOptimizerOptions()
    eqm_opts.niter = 400
    elastic_rods.compute_equilibrium(linkage, fixedVars=fixedVars, options = eqm_opts)



In [ ]:
def design_parameter_solve(l,regularization_weight = 0.1, smoothing_weight = 1):
    l.set_design_parameter_config(use_restLen = True, use_restKappa = True)
    design_opts = elastic_rods.NewtonOptimizerOptions()
    design_opts.niter = 200
    design_opts.verbose = 10
    elastic_rods.designParameter_solve(l, design_opts, regularization_weight = regularization_weight, smoothing_weight = smoothing_weight)
    l.set_design_parameter_config(use_restLen = True, use_restKappa = True)
    return l

In [ ]:
with so(): design_parameter_solve(linkage, 0, 10)

In [ ]:
view.update(preserveExisting=False)

In [ ]:
with so(): 
    jdo = linkage.dofOffsetForJoint(0)
    fixedVars = list(range(jdo, jdo + 6)) # fix rigid motion for a single joint
    elastic_rods.compute_equilibrium(linkage, fixedVars=fixedVars)
view.update(preserveExisting=False)

In [ ]:
export_linkage_geometry_to_obj(linkage, '{}_linkage.obj'.format(MODEL_NAME))

In [ ]:
la.norm(linkage.gradient())

In [ ]:
ribbons = order_segments_by_ribbons(linkage)

In [ ]:
def highlight_rod_and_joint(strip_index, select_joint_index):
    new_rod, fixedVars = construct_elastic_rod_loop_from_rod_segments(linkage, ribbons[strip_index])
    # Set the material of the new rod to be the same as previously.
    new_rod.setMaterial(elastic_rods.RodMaterial('rectangle', 2000, 0.3, RIBBON_CS, stiffAxis=elastic_rods.StiffAxis.D1))
    single_rod_view_compare = linkage_vis.LinkageViewer(linkage, width=1024, height=640)
    single_rod_view = linkage_vis.LinkageViewer(new_rod, width=1024, height=640)

    j = linkage.joint(select_joint_index)
    seg_index = j.segments_A[0]
    vx_index = 0
    if linkage.segment(seg_index).startJoint != select_joint_index:
        vx_index = -1
    joint_vector_field = [np.zeros_like(np.reshape(s.rod.gradient()[0:3*s.rod.numVertices()], (s.rod.numVertices(), 3))) for s in linkage.segments()]
    joint_vector_field[seg_index][vx_index] = linkage.segment(seg_index).rod.deformedConfiguration().materialFrame[vx_index].d2
    single_rod_view_compare.update(vectorField=joint_vector_field)

    sf = vis.fields.ScalarField(new_rod, 0.6 * np.ones_like(np.array(new_rod.deformedConfiguration().len)), colormap=cm.Blues, vmin = 0, vmax = 1)

    single_rod_view_compare.update(mesh = single_rod_view.mesh, preserveExisting=True, scalarField=sf)
    single_rod_view_compare.setCameraParams(((0.1380416750325228, 0.9648987923360046, 4.776431269112697),
     (0.9983340296894934, -0.054896765875897646, -0.01776260848808606),
     (0.0, 0.0, 0.0)))
    return single_rod_view_compare

In [ ]:
single_rod_view_compare = highlight_rod_and_joint(0, 137)
single_rod_view_compare.show()

In [ ]:
# single_rod_view_compare = linkage_vis.LinkageViewer(linkage, width=1024, height=640)
# joint_vector_field = [np.zeros_like(np.reshape(s.rod.gradient()[0:3*s.rod.numVertices()], (s.rod.numVertices(), 3))) for s in linkage.segments()]
# # joint_vector_field = [np.array([frame.d2 for frame in s.rod.deformedConfiguration().materialFrame]) for s in linkage.segments()]


# select_index = 107
# j = linkage.joint(select_index)
# seg_index = j.segments_A[0]
# vx_index = 0
# if linkage.segment(seg_index).startJoint != select_index:
#     vx_index = -1
# joint_vector_field[seg_index][vx_index] = linkage.segment(seg_index).rod.deformedConfiguration().materialFrame[vx_index].d2

# single_rod_view_compare.update(vectorField=joint_vector_field)
# single_rod_view_compare.show()

In [ ]:
# select_index = 106
# j = linkage.joint(select_index)
# seg_index = j.segments_A[0]
# vx_index = 0
# if linkage.segment(seg_index).startJoint != select_index:
#     vx_index = -1
# joint_vector_field[seg_index][vx_index] = linkage.segment(seg_index).rod.deformedConfiguration().materialFrame[vx_index].d2
# single_rod_view_compare.update(vectorField=joint_vector_field)




In [ ]:
# select_index = 96
# j = linkage.joint(select_index)
# seg_index = j.segments_A[0]
# vx_index = 0
# if linkage.segment(seg_index).startJoint != select_index:
#     vx_index = -1
# joint_vector_field[seg_index][vx_index] = linkage.segment(seg_index).rod.deformedConfiguration().materialFrame[vx_index].d2

# single_rod_view_compare.update(vectorField=joint_vector_field)



In [ ]:
# select_index = 132
# j = linkage.joint(select_index)
# seg_index = j.segments_A[0]
# vx_index = 0
# if linkage.segment(seg_index).startJoint != select_index:
#     vx_index = -1
# joint_vector_field[seg_index][vx_index] = linkage.segment(seg_index).rod.deformedConfiguration().materialFrame[vx_index].d2

# single_rod_view_compare.update(vectorField=joint_vector_field)




In [ ]:
linkage.numJoints()

In [ ]:
# with so(): start_linkage, start_view = initialize_linkage(model_path = MODEL_PATH, cross_section = ISO_CS, subdivision_res=SUBDIVISION_RESOLUTION)
# compare_turning_angle(start_linkage, esp = 0, remove_zeros = True)
# start_view.show()

In [ ]:
# linkage = copy.deepcopy(start_linkage)
# linkage.set_design_parameter_config(use_restLen = True, use_restKappa = False)
# # with so():
# #     elastic_rods.restlen_solve(curr_linkage)
# #     elastic_rods.compute_equilibrium(curr_linkage, fixedVars=curr_linkage.jointPositionDoFIndices())
# # view = linkage_vis.LinkageViewer(curr_linkage, width=1024, height=640)
# # view.setCameraParams(default_camera_parameters)
# # view.update(scalarField=get_distance_to_center_scalar_field(curr_linkage))
# # view.show()

In [ ]:
# with so():
#     for i in range(20):
#         linkage = update_rest_curvature(linkage)
# #     Visualization
# #     compare_turning_angle(curr_linkage, rest_linkage = prev_linkage)
# #     prev_linkage = copy.deepcopy(curr_linkage)
# #     view.update(scalarField=get_distance_to_center_scalar_field(curr_linkage))
# #     input("Press Enter to continue...")




In [ ]:
get_all_curve_pattern(linkage, RIBBON_CS[1], SUBDIVISION_RESOLUTION, RIBBON_NAME, image_type='svg')